## Statistical Analysis in Python

In this section, we introduce a few useful methods for analyzing your data in Python.
Namely, we cover how to compute the mean, variance, and standard error of a data set.
For more advanced statistical analysis, we cover how to perform a
Mann-Whitney-Wilcoxon (MWW) RankSum test, how to perform an Analysis of variance (ANOVA)
between multiple data sets, and how to compute bootstrapped 95% confidence intervals for
non-normally distributed data sets.

### Python's SciPy Module

The majority of data analysis in Python can be performed with the SciPy module. SciPy
provides a plethora of statistical functions and tests that will handle the majority of
your analytical needs. If we don't cover a statistical function or test that you require
for your research, SciPy's full statistical library is described in detail at:
http://docs.scipy.org/doc/scipy/reference/tutorial/stats.html

### Python's pandas Module

The pandas module provides powerful, efficient, R-like DataFrame objects capable of
calculating statistics en masse on the entire DataFrame. DataFrames are useful for when
you need to compute statistics over multiple replicate runs.

For the purposes of this tutorial, we will use Luis Zaman's parasite data set (available from https://raw.github.com/briandconnelly/BEACONToolkit/master/analysis/data/parasite_data.csv):

In [1]:
from pandas import *

# must specify that blank space " " is NaN
experimentDF = read_csv("../DATA/parasite_data.csv", na_values=[" "])

print(experimentDF)

     Virulence  Replicate  ShannonDiversity
0          0.5          1          0.059262
1          0.5          2          1.093600
2          0.5          3          1.139390
3          0.5          4          0.547651
4          0.5          5          0.065928
5          0.5          6          1.344330
6          0.5          7          1.680480
7          0.5          8          0.000000
8          0.5          9          2.047680
9          0.5         10          0.000000
10         0.5         11          1.507140
11         0.5         12          0.000000
12         0.5         13          1.589810
13         0.5         14          1.144800
14         0.5         15          1.011190
15         0.5         16          0.000000
16         0.5         17          0.776665
17         0.5         18          0.001749
18         0.5         19          1.761200
19         0.5         20          0.021091
20         0.5         21          0.790915
21         0.5         22       

### Accessing data in pandas DataFrames

You can directly access any column and row by indexing the DataFrame.

In [2]:
# show all entries in the Virulence column
print(experimentDF["Virulence"])

0      0.5
1      0.5
2      0.5
3      0.5
4      0.5
5      0.5
6      0.5
7      0.5
8      0.5
9      0.5
10     0.5
11     0.5
12     0.5
13     0.5
14     0.5
15     0.5
16     0.5
17     0.5
18     0.5
19     0.5
20     0.5
21     0.5
22     0.5
23     0.5
24     0.5
25     0.5
26     0.5
27     0.5
28     0.5
29     0.5
      ... 
320    NaN
321    NaN
322    NaN
323    NaN
324    NaN
325    NaN
326    NaN
327    NaN
328    NaN
329    NaN
330    NaN
331    NaN
332    NaN
333    NaN
334    NaN
335    NaN
336    NaN
337    NaN
338    NaN
339    NaN
340    NaN
341    NaN
342    NaN
343    NaN
344    NaN
345    NaN
346    NaN
347    NaN
348    NaN
349    NaN
Name: Virulence, dtype: float64


In [3]:
# show the 12th row in the ShannonDiversity column
print(experimentDF["ShannonDiversity"][12])

1.58981


You can also access all of the values in a column meeting a certain criteria.

In [4]:
# show all entries in the ShannonDiversity column > 2.0
print(experimentDF[experimentDF["ShannonDiversity"] > 2.0])

     Virulence  Replicate  ShannonDiversity
8          0.5          9           2.04768
89         0.6         40           2.01066
92         0.6         43           2.90081
96         0.6         47           2.02915
105        0.7          6           2.23427
117        0.7         18           2.14296
127        0.7         28           2.23599
129        0.7         30           2.48422
133        0.7         34           2.18506
134        0.7         35           2.42177
139        0.7         40           2.25737
142        0.7         43           2.07258
148        0.7         49           2.38326
151        0.8          2           2.07970
153        0.8          4           2.38474
163        0.8         14           2.03252
165        0.8         16           2.38415
170        0.8         21           2.02297
172        0.8         23           2.13882
173        0.8         24           2.53339
182        0.8         33           2.17865
196        0.8         47       

### Blank/omitted data (NA or NaN) in pandas DataFrames

Blank/omitted data is a piece of cake to handle in pandas. Here's an example data
set with NA/NaN values.

In [5]:
print(experimentDF[notnull(experimentDF["Virulence"])])

     Virulence  Replicate  ShannonDiversity
0          0.5          1          0.059262
1          0.5          2          1.093600
2          0.5          3          1.139390
3          0.5          4          0.547651
4          0.5          5          0.065928
5          0.5          6          1.344330
6          0.5          7          1.680480
7          0.5          8          0.000000
8          0.5          9          2.047680
9          0.5         10          0.000000
10         0.5         11          1.507140
11         0.5         12          0.000000
12         0.5         13          1.589810
13         0.5         14          1.144800
14         0.5         15          1.011190
15         0.5         16          0.000000
16         0.5         17          0.776665
17         0.5         18          0.001749
18         0.5         19          1.761200
19         0.5         20          0.021091
20         0.5         21          0.790915
21         0.5         22       

DataFrame methods automatically ignore NA/NaN values.

In [6]:
print("Mean virulence across all treatments:", experimentDF["Virulence"].mean())

Mean virulence across all treatments: 0.7500000000000013


However, not all methods in Python are guaranteed to handle NA/NaN values properly.

In [7]:
from scipy import stats

print("Mean virulence across all treatments:", stats.sem(experimentDF["Virulence"]))

Mean virulence across all treatments: nan


Thus, it behooves you to take care of the NA/NaN values before performing your analysis. You can either:

**(1) filter out all of the entries with NA/NaN**

In [8]:
# NOTE: this drops the entire row if any of its entries are NA/NaN!
print(experimentDF.dropna())

     Virulence  Replicate  ShannonDiversity
0          0.5          1          0.059262
1          0.5          2          1.093600
2          0.5          3          1.139390
3          0.5          4          0.547651
4          0.5          5          0.065928
5          0.5          6          1.344330
6          0.5          7          1.680480
7          0.5          8          0.000000
8          0.5          9          2.047680
9          0.5         10          0.000000
10         0.5         11          1.507140
11         0.5         12          0.000000
12         0.5         13          1.589810
13         0.5         14          1.144800
14         0.5         15          1.011190
15         0.5         16          0.000000
16         0.5         17          0.776665
17         0.5         18          0.001749
18         0.5         19          1.761200
19         0.5         20          0.021091
20         0.5         21          0.790915
21         0.5         22       

If you only care about NA/NaN values in a specific column, you can specify the
column name first.

In [9]:
print(experimentDF["Virulence"].dropna())

0      0.5
1      0.5
2      0.5
3      0.5
4      0.5
5      0.5
6      0.5
7      0.5
8      0.5
9      0.5
10     0.5
11     0.5
12     0.5
13     0.5
14     0.5
15     0.5
16     0.5
17     0.5
18     0.5
19     0.5
20     0.5
21     0.5
22     0.5
23     0.5
24     0.5
25     0.5
26     0.5
27     0.5
28     0.5
29     0.5
      ... 
270    1.0
271    1.0
272    1.0
273    1.0
274    1.0
275    1.0
276    1.0
277    1.0
278    1.0
279    1.0
280    1.0
281    1.0
282    1.0
283    1.0
284    1.0
285    1.0
286    1.0
287    1.0
288    1.0
289    1.0
290    1.0
291    1.0
292    1.0
293    1.0
294    1.0
295    1.0
296    1.0
297    1.0
298    1.0
299    1.0
Name: Virulence, dtype: float64


**(2) replace all of the NA/NaN entries with a valid value**

In [10]:
print(experimentDF.fillna(0.0)["Virulence"])

0      0.5
1      0.5
2      0.5
3      0.5
4      0.5
5      0.5
6      0.5
7      0.5
8      0.5
9      0.5
10     0.5
11     0.5
12     0.5
13     0.5
14     0.5
15     0.5
16     0.5
17     0.5
18     0.5
19     0.5
20     0.5
21     0.5
22     0.5
23     0.5
24     0.5
25     0.5
26     0.5
27     0.5
28     0.5
29     0.5
      ... 
320    0.0
321    0.0
322    0.0
323    0.0
324    0.0
325    0.0
326    0.0
327    0.0
328    0.0
329    0.0
330    0.0
331    0.0
332    0.0
333    0.0
334    0.0
335    0.0
336    0.0
337    0.0
338    0.0
339    0.0
340    0.0
341    0.0
342    0.0
343    0.0
344    0.0
345    0.0
346    0.0
347    0.0
348    0.0
349    0.0
Name: Virulence, dtype: float64


Take care when deciding what to do with NA/NaN entries. It can have a significant
impact on your results!

In [11]:
print("Mean virulence across all treatments w/ dropped NaN:", experimentDF["Virulence"].dropna().mean())

print("Mean virulence across all treatments w/ filled NaN:", experimentDF.fillna(0.0)["Virulence"].mean())

Mean virulence across all treatments w/ dropped NaN: 0.7500000000000013
Mean virulence across all treatments w/ filled NaN: 0.642857142857144


### Mean

The mean performance of an experiment gives a good idea of how the experiment will
turn out *on average* under a given treatment.

Conveniently, DataFrames have all kinds of built-in functions to perform standard
operations on them en masse: `add()`, `sub()`, `mul()`, `div()`, `mean()`, `std()`, etc.
The full list is located at: http://pandas.sourceforge.net/generated/pandas.DataFrame.html

Thus, computing the mean of a DataFrame only takes one line of code:

In [12]:
from pandas import *

print("Mean Shannon Diversity w/ 0.8 Parasite Virulence =", experimentDF[experimentDF["Virulence"] == 0.8]["ShannonDiversity"].mean())

Mean Shannon Diversity w/ 0.8 Parasite Virulence = 1.2691338187999996


### Variance

The variance in the performance provides a measurement of how consistent the results
of an experiment are. The lower the variance, the more consistent the results are, and
vice versa.

Computing the variance is also built in to pandas DataFrames:

In [13]:
from pandas import *

print "Variance in Shannon Diversity w/ 0.8 Parasite Virulence =", experimentDF[experimentDF["Virulence"] == 0.8]["ShannonDiversity"].var()

SyntaxError: invalid syntax (<ipython-input-13-9d187b3a8556>, line 3)

### Standard Error of the Mean (SEM)

Combined with the mean, the SEM enables you to establish a range around a mean that
the majority of any future replicate experiments will most likely fall within.

pandas DataFrames don't have methods like SEM built in, but since DataFrame
rows/columns are treated as lists, you can use any NumPy/SciPy method you like on them.

In [ ]:
from pandas import *
from scipy import stats

print("SEM of Shannon Diversity w/ 0.8 Parasite Virulence =", stats.sem(experimentDF[experimentDF["Virulence"] == 0.8]["ShannonDiversity"]))

A single SEM will usually envelop 68% of the possible replicate means
and two SEMs envelop 95% of the possible replicate means. Two
SEMs are called the "estimated 95% confidence interval." The confidence
interval is estimated because the exact width depend on how many replicates
you have; this approximation is good when you have more than 20 replicates.

### Mann-Whitney-Wilcoxon (MWW) RankSum test

The MWW RankSum test is a useful test to determine if two distributions are significantly
different or not. Unlike the t-test, the RankSum test does not assume that the data
are normally distributed, potentially providing a more accurate assessment of the data sets.

As an example, let's say we want to determine if the results of the two following
treatments significantly differ or not:

In [ ]:
# select two treatment data sets from the parasite data
treatment1 = experimentDF[experimentDF["Virulence"] == 0.5]["ShannonDiversity"]
treatment2 = experimentDF[experimentDF["Virulence"] == 0.8]["ShannonDiversity"]

print("Data set 1:\n", treatment1)
print("Data set 2:\n", treatment2)

A RankSum test will provide a P value indicating whether or not the two
distributions are the same.

In [ ]:
from scipy import stats

z_stat, p_val = stats.ranksums(treatment1, treatment2)

print("MWW RankSum P for treatments 1 and 2 =", p_val)

If P <= 0.05, we are highly confident that the distributions significantly differ, and
can claim that the treatments had a significant impact on the measured value.

If the treatments do *not* significantly differ, we could expect a result such as
the following:

In [ ]:
treatment3 = experimentDF[experimentDF["Virulence"] == 0.8]["ShannonDiversity"]
treatment4 = experimentDF[experimentDF["Virulence"] == 0.9]["ShannonDiversity"]

print("Data set 3:\n", treatment3)
print("Data set 4:\n", treatment4)

In [ ]:
z_stat, p_val = stats.ranksums(treatment3, treatment4)

print("MWW RankSum P for treatments 3 and 4 =", p_val)

With P > 0.05, we must say that the distributions do not significantly differ.
Thus changing the parasite virulence between 0.8 and 0.9 does not result in a
significant change in Shannon Diversity.

### One-way analysis of variance (ANOVA)

If you need to compare more than two data sets at a time, an ANOVA is your best bet. For
example, we have the results from three experiments with overlapping 95% confidence
intervals, and we want to confirm that the results for all three experiments are not
significantly different.

In [ ]:
treatment1 = experimentDF[experimentDF["Virulence"] == 0.7]["ShannonDiversity"]
treatment2 = experimentDF[experimentDF["Virulence"] == 0.8]["ShannonDiversity"]
treatment3 = experimentDF[experimentDF["Virulence"] == 0.9]["ShannonDiversity"]

print("Data set 1:\n", treatment1)
print("Data set 2:\n", treatment2)
print("Data set 3:\n", treatment3)

In [ ]:
from scipy import stats
	
f_val, p_val = stats.f_oneway(treatment1, treatment2, treatment3)

print("One-way ANOVA P =", p_val)

If P > 0.05, we can claim with high confidence that the means of the results of all three
experiments are not significantly different.

### Bootstrapped 95% confidence intervals

Oftentimes in wet lab research, it's difficult to perform the 20 replicate runs
recommended for computing reliable confidence intervals with SEM.

In this case, bootstrapping the confidence intervals is a much more accurate method of
determining the 95% confidence interval around your experiment's mean performance.

Unfortunately, SciPy doesn't have bootstrapping built into its standard library yet.
However, there is already a scikit out there for bootstrapping. Enter the following
command to install it:

> sudo pip install -e git+http://github.org/cgevans/scikits-bootstrap.git#egg=Package

You may need to install `pip` first, e.g., for Mac:

> sudo easy_install pip

Bootstrapping 95% confidence intervals around the mean with this function is simple:

In [ ]:
# subset a list of 10 data points
treatment1 = experimentDF[experimentDF["Virulence"] == 0.8]["ShannonDiversity"][:10]

print("Small data set:\n", treatment1)

In [ ]:
import scipy
import scikits.bootstrap as bootstrap

CIs = bootstrap.ci(data=treatment1, statfunction=scipy.mean)

print("Bootstrapped 95% confidence intervals\nLow:", CIs[0], "\nHigh:", CIs[1])

Note that you can change the range of the confidence interval by setting the alpha:

In [ ]:
# 80% confidence interval
CIs = bootstrap.ci(treatment1, scipy.mean, alpha=0.2)
print("Bootstrapped 80% confidence interval\nLow:", CIs[0], "\nHigh:", CIs[1])

And also modify the size of the bootstrapped sample pool that the confidence intervals
are taken from:

In [ ]:
# bootstrap 20,000 samples instead of only 10,000
CIs = bootstrap.ci(treatment1, scipy.mean, n_samples=20000)
print("Bootstrapped 95% confidence interval w/ 20,000 samples\nLow:", CIs[0], "\nHigh:", CIs[1])

Generally, bootstrapped 95% confidence intervals provide more accurate confidence
intervals than 95% confidence intervals estimated from the SEM.